In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install transformers

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
file_path = "/content/drive/MyDrive/my-datasets/yektanet_train.csv"

data = pd.read_csv(file_path)

# data Preprocessing
def preprocess_data(df):
   # Get unique categories
    categories = df['category'].unique()

    # Create new dataframe with all columns except 'category'
    new_df = df.copy()

    # Create binary columns for each category
    for category in categories:
        # Create new column with category name and fill with 0s and 1s
        new_df[category] = (df['category'] == category).astype(int)

    # Drop the original category column
    new_df = new_df.drop('category', axis=1)

    return new_df

proccessed_data = preprocess_data(data)

# get all categories
non_category_columns = ['description', 'text_content', 'title']
category_columns = [col for col in proccessed_data.columns if col not in non_category_columns]

proccessed_data["content"] = proccessed_data['title'] + "-" + proccessed_data['text_content'] + "-" + proccessed_data['description']


# drop useless columns
cols_to_drop = ["title", "text_content","description","url", "domain", "id","h1","h2"]
existing_cols = [col for col in cols_to_drop if col in proccessed_data.columns]
proccessed_data.drop(labels=existing_cols, axis=1, inplace=True)

proccessed_data.head()


,کتاب و ادبیات,تجارت و اقتصاد,سلامت,تکنولوژی و کامپبوتر,ورزش,غذا و نوشیدنی,حقوق و دولت و سیاست,مسکن,خودرو,مد و زیبایی,...,سفر و گردشگری,هنر و سرگرمی,خانه و باغبانی,موسیقی,تحصیلات,علم و دانش,خانواده,اشتغال,حیوانات خانگی,content
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,معنی از شوبنده ها | جدول یاب- معنی از شوبنده ...
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,عکس بیت‌کوین کش برای پروفایل-عکس بیت‌کوین کش ب...
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,دکتر مهناز عابدینی متخصص رادیولوژی و سونوگرافی...
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,دانلود تحریم‌گذر Geph برای اندروید-دانلود تحری...
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ترفندهای پرکاربرد تلویزیون‌‌های هوشمند سامسونگ...


In [21]:
features = proccessed_data['content']

target_list = proccessed_data.drop('content',axis=1)

x_train, x_test, y_train, y_test = train_test_split(
    features, target_list, test_size=0.2, random_state=42, stratify=target_list
)

print("Training set shape:", x_train.shape)
print("Test set shape:", x_test.shape)

# print("Training set shape:", y_train.shape)
# print("Test set shape:", y_test.shape)

Training set shape: (3831,)
Test set shape: (958,)


In [22]:
# implement hypo parameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

In [23]:
from transformers import BertTokenizer,BertModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [24]:
# [CLS] , [PAD] , [SEP] , [MASK]

example_text = "My name is hossein"

encodings = tokenizer.encode_plus(
    example_text,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

encodings

{'input_ids': tensor([[  101,  2026,  2171,  2003,  7570, 11393,  2378,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [29]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.title = df['content'].astype(str)
         # Get all category columns (assuming they start with 'category_')
        self.category_columns = [col for col in df.columns if col.startswith('category_')]
        self.targets = df[self.category_columns].values

    def __len__(self):
        return len(self.title)

    def __getitem__(self,index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(), # [1, 512] => [512]
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
        }

In [30]:
train_size = 0.8

print(x_train)

train_df = pd.concat([x_train, y_train], axis=1)
train_df = train_df.sample(frac=train_size,random_state=200).reset_index(drop=True)

val_df = pd.concat([x_test, y_test], axis=1)
val_df = val_df.drop(x_test.index).reset_index(drop=True)

3380    Mamad ahmadi-Mamad ahmadi Skip to content شبکه...
1457    خواص عاقرقرحا ؛ نحوه استفاده از عاقرقرحا در طب...
1473    روزنامه کيهان:                                ...
3316    آغاز ثبت نام طرح‌های فروش محصولات ایران‌خودرو-...
148     تور خارجی | ترکیه 🕌 ، امارات ، تایلند ، مالزی ...
                              ...                        
2861    روش درست برخورد با بیماری همسر چیست؟-روش درست ...
3314    یاسمین مقبلی، زن ایرانی که به ماه می‌رود - بها...
4104    بذر کاکتوس های ستونی میکس بسته ۱۰ عددی - میهن ...
4519    خرید و قیمت کتاب قصه ها و غصه های زال زر اثر ع...
3832    ماهنامه شمارۀ 50 و 51 پیش به سوی کشاورزی اقتصا...
Name: content, Length: 3831, dtype: object


In [31]:
train_dataset = CustomDataset(train_df,tokenizer,MAX_LEN)
valid_dataset = CustomDataset(val_df,tokenizer,MAX_LEN)

In [32]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=TRAIN_BATCH_SIZE,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    shuffle=False,
    batch_size=VALID_BATCH_SIZE,
    num_workers=0
)

In [33]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


In [34]:
def load_ckp(checkpoint_fpath,model,optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'],valid_loss_min.item()


def save_ckp(state,is_best,checkpoint_path,best_model_path):
    f_path = checkpoint_path
    torch.save(state,f_path)
    if is_best:
        best_fpath = best_model_path
        shutil.copyfile(f_path,best_fpath)

In [35]:
class BERTClass(nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased',return_dict=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768,6)

    def forward(self,input_ids,attention_mask,token_type_ids):
      output = self.bert_model(input_ids,attention_mask,token_type_ids)
      output_dropout = self.dropout(output.pooler_output)
      output = self.linear(output_dropout)
      return output

model = BERTClass()
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [36]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs,targets)

optimizer = torch.optim.Adam(params=model.parameters(),lr=LEARNING_RATE)

In [37]:
def train_model(
    n_epochs,
    training_loader,
    validation_loader,
    model,
    optimizer,
    checkpoint_path,
    best_model_path
):
    valid_loss_min = np.Inf

    for epoch in range(1, n_epochs+1):
        train_loss = 0
        valid_loss = 0
        model.train()

        # training loop
        for index, batch in enumerate(training_loader):
            input_ids = batch['input_ids'].to(device, dtype=torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
            targets = batch['targets'].to(device, dtype=torch.float)
            outputs = model(input_ids,attention_mask,token_type_ids)
            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss = train_loss + (1/(index+1) (loss.item() - train_loss))

        # validation loop
        model.eval()
        with torch.no_grad():
             for index, batch in enumerate(validation_loader):
                 input_ids = batch['input_ids'].to(device, dtype=torch.long)
                 attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
                 token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
                 targets = batch['targets'].to(device, dtype=torch.float)
                 outputs = model(input_ids,attention_mask,token_type_ids)
                 optimizer.zero_grad()
                 loss = loss_fn(outputs, targets)
                 optimizer.step()
                 val_loss = valid_loss + (1/(index+1) (loss.item() - valid_loss))

        checkpoint = {
             'epoch' : epoch+1,
             'valid_loss_min': valid_loss,
             'state_dict': model.state_dict(),
             'optimizer': optimizer.state_dict()
            }

        save_ckp(checkpoint,False,checkpoint_path,best_model_path)

    return model

In [59]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, "/curr_ckpt","/best.pt")

ValueError: Target size (torch.Size([32, 0])) must be the same as input size (torch.Size([32, 6]))

In [58]:
example = x_test.iloc[300]
print(example)

encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

model.eval()
with torch.no_grad():
     input_ids = encodings['input_ids'].to(device, dtype=torch.long)
     attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
     token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
     output = model(input_ids,attention_mask,token_type_ids)
     final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
     print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

هنرستان پسرانه شهیدمفتح دماوند-هنرستان پسرانه شهیدمفتح دماوند دانش‌نامه مدارس برگ نخست استان مدرسه دولتی پسرانه شهیدمفتح (دماوند) در مقطع کاردانش شامل رشته های برق ساختمان، تولید و توسعه دهنده‌ی پایگاه‌های اینترنتی و طراحی و توسعه‌ی صفحات وب شهری شبانه‌روزی تماس هم‌رسانی معرفی هنرستان پسرانه شهیدمفتح از جمله بهترین مدارس دولتی واقع در شهر دماوند استان تهران می‌باشد. این مرکز به دانش‌آموزان کاردانش در رشته‌ها‌ی برق ساختمان، تولید و توسعه دهنده‌ی پایگاه‌های اینترنتی و طراحی و توسعه‌ی صفحات وب، خدمات آموزشی ارائه می‌نماید. مدیریت مجموعه را سعید لطیفی بر عهده دارد. اولیای گرامی دانش‌آموزان دماوند می‌توانند جهت مشاوره و نیز انجام امور ثبت‌ نام به صورت حضوری با مراجعه به نشانی کیلان کوهان جنب پمپ گاز؛ کدپستی 3976113585 یا غیر حضوری از طریق تماس تلفنی با 76362288 (پیش‌شماره: 021) اقدام فرمایند. امکانات رفاهی سرویس ایاب و ذهاب سالن غذاخوری بوفه کمد شخصی اتاق بازی نمازخانه چمن مصنوعی ساختمان مساحت زمین مساحت بنا تعداد طبقه سال ساخت تعداد کلاس نما تجهیزات دوربین دستگاه ثبت تردد تخته هوشمند ویدئو

<ipython-input-58-92be12ea825a>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
